# Bibliotecas 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
from osgeo import gdal
import cv2
import os
import time
import mlflow
from mlflow import MlflowClient
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

# Pré-processamento do Dataset

In [2]:
def load_and_crop_data(image_dir, mask_dir, crop_size, augment=False, augment_factor=4):
    images = []
    masks = []

    # Listar arquivos nas pastas de imagens e máscaras
    image_files = sorted(os.listdir(image_dir))
    mask_files = sorted(os.listdir(mask_dir))

    # Garantir que a correspondência entre imagens e máscaras seja correta
    assert len(image_files) == len(mask_files), "Número de imagens e máscaras deve ser o mesmo."
    
    for img_name, mask_name in zip(image_files, mask_files):
        img_path = os.path.join(image_dir, img_name)
        mask_path = os.path.join(mask_dir, mask_name)

        # Abrir a imagem e a máscara
        img_ds = gdal.Open(img_path)
        mask_ds = gdal.Open(mask_path)

        if img_ds is None or mask_ds is None:
            raise FileNotFoundError(f"Erro ao abrir imagem ou máscara: {img_path}, {mask_path}")

        # Verificar se as dimensões da imagem e máscara coincidem
        assert img_ds.RasterXSize == mask_ds.RasterXSize and img_ds.RasterYSize == mask_ds.RasterYSize, \
            f"Dimensões diferentes para imagem ({img_name}) e máscara ({mask_name})."

        img_width, img_height = img_ds.RasterXSize, img_ds.RasterYSize
        img_bands = img_ds.RasterCount

        # Iterar pelos blocos na imagem e máscara
        for y in range(0, img_height - crop_size + 1, crop_size):
            for x in range(0, img_width - crop_size + 1, crop_size):

                # Ler bloco da imagem
                img_block = img_ds.ReadAsArray(x, y, crop_size, crop_size) # (4, 128, 128)

                # --- CORREÇÃO AQUI ---
                # Garanta que img_block esteja sempre no formato (H, W, C)
                if img_block.ndim == 3: # Se for multibanda (C, H, W)
                    # Transpor para (H, W, C)
                    img_block = np.transpose(img_block, (1, 2, 0))

                elif img_block.ndim == 2: # Se for escala de cinza (H, W)
                    # Adicionar dimensão de canal para consistência
                    img_block = np.expand_dims(img_block, axis=-1)
                # --- FIM DA CORREÇÃO ---

                # --- ADICIONE ESTA LINHA ---
                # Se o bloco tiver 4 canais, mantenha apenas os 3 primeiros (RGB)
                if img_block.shape[2] == 4:
                    img_block = img_block[:, :, :3]  # Fatiando para pegar os canais 0, 1 e 2
                # --- FIM DA LINHA ADICIONADA ---

                # Ler bloco da máscara
                mask_block = mask_ds.ReadAsArray(x, y, crop_size, crop_size)
                
                # --- CORREÇÃO APLICADA AQUI ---
                # Padroniza a forma da máscara para (H, W) antes de adicionar o canal
                if mask_block.ndim == 3:
                    # Se a forma for (1, H, W), remove a primeira dimensão para ficar (H, W)
                    mask_block = np.squeeze(mask_block, axis=0)

                # Garante que a máscara tenha um canal no final, resultando em (H, W, 1)
                if mask_block.ndim == 2:
                    mask_block = np.expand_dims(mask_block, axis=-1)
                # --- FIM DA CORREÇÃO ---

                # Normalizar os blocos
                img_block = (img_block / 255.0).astype(np.float32)
                mask_block = mask_block.astype(np.float32)
                mask_block[mask_block > 0] = 1
                # Adicionar canal extra
                #mask_block = np.expand_dims(mask_block, axis=-1)

                # Adicionar os blocos às listas
                images.append(img_block)
                masks.append(mask_block)

                # Aplicar aumentação se solicitado
                if augment:
                    # A função agora retorna uma lista com 4 novas versões
                    list_of_aug_images, list_of_aug_masks = augment_data(img_block, mask_block)

                    # Usa .extend() para adicionar todas as novas versões de uma vez
                    images.extend(list_of_aug_images)
                    masks.extend(list_of_aug_masks)

    return np.array(images), np.array(masks)

# Função para realizar data augmentation
def augment_data(image, mask):
    augmented_images = []
    augmented_masks = []

    # 1. Flip Horizontal (espelhamento)
    augmented_images.append(np.fliplr(image))
    augmented_masks.append(np.fliplr(mask))

    # 2. Flip Vertical (de cabeça para baixo)
    augmented_images.append(np.flipud(image))
    augmented_masks.append(np.flipud(mask))

    # 3. Rotação de 90 graus (sentido anti-horário)
    augmented_images.append(np.rot90(image, k=1))
    augmented_masks.append(np.rot90(mask, k=1))
    
    # 4. Rotação de 270 graus (ou -90 graus)
    augmented_images.append(np.rot90(image, k=3))
    augmented_masks.append(np.rot90(mask, k=3))
    
    return augmented_images, augmented_masks

# U-Net simplificada

In [ ]:
# U-Net simplificada
def build_unet(input_shape):
    inputs = layers.Input(shape=input_shape)

    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)

    u1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c3)
    u1 = layers.concatenate([u1, c2])
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    u2 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c4)
    u2 = layers.concatenate([u2, c1])
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c5)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c5)

    return models.Model(inputs, outputs)

# --- COMO USAR ---

# 1. Construa o modelo
input_shape = (128, 128, 3)
model = build_unet(input_shape=input_shape)

# 3. Veja o resultado
model.summary()

# U-Net com encoder pré-treinado

In [ ]:
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Input

def build_unet_mobilenetv2(input_shape, num_classes=1):
    """
    Constrói uma arquitetura U-Net usando um encoder VGG16 pré-treinado.

    Args:
        input_shape (tuple): O tamanho dos patches de entrada (altura, largura, canais).
        num_classes (int): O número de classes de saída. Para segmentação binária, use 1.

    Returns:
        keras.Model: O modelo U-Net compilado.
    """
    
    # 1. CARREGAR O ENCODER (BACKBONE) VGG16 PRÉ-TREINADO
    # include_top=False remove as camadas de classificação no final.
    # weights='imagenet' carrega os pesos aprendidos com o dataset ImageNet.
    backbone = keras.applications.MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )

    # Congelar os pesos do encoder para que eles não sejam treinados inicialmente.
    # Vamos apenas treinar nosso novo decoder.
    backbone.trainable = False

    # 2. IDENTIFICAR AS CAMADAS DE SKIP CONNECTION DO ENCODER
    # Precisamos das saídas das camadas de Max-Pooling do VGG16 para conectar ao decoder.
    # Você pode ver os nomes das camadas rodando `backbone.summary()`.
    skip_connections_names = [
        'block_1_expand_relu',  # 64x64
        'block_3_expand_relu',  # 32x32
        'block_6_expand_relu',  # 16x16
        'block_13_expand_relu',  # 8x8
    ]
    # Pega a saída (tensor) de cada uma dessas camadas.
    encoder_outputs = [backbone.get_layer(name).output for name in skip_connections_names]
    
    # A entrada para o decoder será a saída final do encoder.
    encoder_final_output = backbone.output # 4x4

    # 3. CONSTRUIR O DECODER (CAMINHO DE EXPANSÃO)
    # Vamos subir, aumentando a resolução e concatenando com as skip connections.
    
    # Bloco expansivo 1
    # Sobe de 4x4 para 8x8
    up_stack_1 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(encoder_final_output)
    concat_1 = Concatenate()([up_stack_1, encoder_outputs[3]]) # Conecta com a saída do block4_pool
    conv_stack_1 = Conv2D(128, 3, activation='relu', padding='same')(concat_1)
    conv_stack_1 = Conv2D(128, 3, activation='relu', padding='same')(conv_stack_1)

    # Bloco expansivo 2
    # Sobe de 8x8 para 16x16
    up_stack_2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv_stack_1)
    concat_2 = Concatenate()([up_stack_2, encoder_outputs[2]]) # Conecta com a saída do block3_pool
    conv_stack_2 = Conv2D(64, 3, activation='relu', padding='same')(concat_2)
    conv_stack_2 = Conv2D(64, 3, activation='relu', padding='same')(conv_stack_2)

    # Bloco expansivo 3
    # Sobe de 16x16 para 32x32
    up_stack_3 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv_stack_2)
    concat_3 = Concatenate()([up_stack_3, encoder_outputs[1]]) # Conecta com a saída do block2_pool
    conv_stack_3 = Conv2D(32, 3, activation='relu', padding='same')(concat_3)
    conv_stack_3 = Conv2D(32, 3, activation='relu', padding='same')(conv_stack_3)

    # Bloco expansivo 4
    # Sobe de 32x32 para 64x64
    up_stack_4 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(conv_stack_3)
    concat_4 = Concatenate()([up_stack_4, encoder_outputs[0]]) # Conecta com a saída do block1_pool
    conv_stack_4 = Conv2D(16, 3, activation='relu', padding='same')(concat_4)
    conv_stack_4 = Conv2D(16, 3, activation='relu', padding='same')(conv_stack_4)
    
    # Bloco expansivo 5 (final para restaurar o tamanho original)
    # Sobe de 64x64 para 128x128
    up_stack_5 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same')(conv_stack_4)
    conv_stack_5 = Conv2D(8, 3, activation='relu', padding='same')(up_stack_5)
    conv_stack_5 = Conv2D(8, 3, activation='relu', padding='same')(conv_stack_5)


    # 4. CAMADA DE SAÍDA
    # Usa um filtro com o número de classes e a ativação apropriada.
    # Para segmentação binária, é 1 classe com ativação 'sigmoid'.
    output_layer = Conv2D(num_classes, 1, activation='sigmoid')(conv_stack_5)

    # 5. CRIAR E RETORNAR O MODELO FINAL
    # A entrada do modelo é a entrada do backbone VGG16.
    model = keras.Model(inputs=backbone.input, outputs=output_layer)
    
    return model

# --- COMO USAR ---

# 1. Construa o modelo
input_shape = (128, 128, 3)
model = build_unet_mobilenetv2(input_shape=input_shape)

# 3. Veja o resultado
model.summary()

# Funções de Treinamento

In [14]:
def iou(y_true, y_pred):
    # Garantir que ambos os tensores sejam do tipo float32
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred > 0.5, tf.float32)  # Binarizar y_pred com threshold de 0.5
    
    # Calcular interseção e união
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    
    # Prevenir divisão por zero
    return tf.math.divide_no_nan(intersection, union)

def dice_coef(y_true, y_pred, threshold=0.5, epsilon=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    y_true_flat = keras.layers.Flatten()(y_true)
    y_pred_flat = keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_flat * y_pred_flat)
    return (2. * intersection + epsilon) / (tf.reduce_sum(y_true_flat) + tf.reduce_sum(y_pred_flat) + epsilon)

def specificity(y_true, y_pred, threshold=0.5, epsilon=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    tn = tf.reduce_sum((1 - y_true) * (1 - y_pred))
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    return tn / (tn + fp + epsilon)

def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice_coefficient = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - dice_coefficient # A perda é 1 - o coeficiente

def weighted_binary_crossentropy(y_true, y_pred, pos_weight=100.):
    y_true = K.cast(y_true, tf.float32)
    
    # Calcula a cross-entropy
    bce = K.binary_crossentropy(y_true, y_pred)
    
    # Aplica os pesos
    weight_vector = y_true * pos_weight + (1. - y_true)
    weighted_bce = weight_vector * bce
    
    return K.mean(weighted_bce)

def combined_loss(y_true, y_pred, alpha=0.3):
    return alpha * weighted_binary_crossentropy(y_true, y_pred) + (1 - alpha) * dice_loss(y_true, y_pred)


def train_model(train_image_dir, train_mask_dir, epochs, crop_size, batch_size, lr, gamma, experiment_name, run_name, model_name, saving_dir, augment):
    # Carregar, recortar e aplicar data augmentation nos dados de treinamento
    print("Carregando, recortando e aplicando data augmentation nos dados de treinamento...")
    start_time = time.time()
    train_images, train_masks = load_and_crop_data(train_image_dir, train_mask_dir, crop_size=crop_size, augment=augment)
    print(f"Dados de treinamento carregados e aumentados em {time.time() - start_time:.2f} segundos")
    print(f"Número total de imagens: {len(train_images)}")

    # Dividir os dados em 70% treino e 30% validação
    train_images, val_images, train_masks, val_masks = train_test_split(
        train_images, train_masks, test_size=0.3, random_state=42
    )

    # Criar datasets para treinamento e validação
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_masks))
    train_dataset = train_dataset.shuffle(len(train_images)).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_masks))
    val_dataset = val_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    print(f"Treinamento: {len(train_images)} imagens")
    print(f"Validação: {len(val_images)} imagens")

    # Construir modelo
    print("Construindo o modelo...")
    input_shape = (crop_size, crop_size, 3)  # Imagens recortadas
    model = build_unet_mobilenetv2(input_shape)
    print("Modelo construído!")

    mlflow.set_tracking_uri('http://localhost:5000')

    # Defina um nome para o seu experimento no MLflow
    mlflow.set_experiment(experiment_name)

    # >>>>> A MÁGICA ACONTECE AQUI <<<<<
    # Ative o autologging para Keras
    mlflow.keras.autolog()

    with mlflow.start_run(run_name=run_name) as run:

        # Defina as métricas que o Keras irá calcular (e o MLflow irá capturar)
        metrics_to_track = [
            keras.metrics.BinaryIoU(threshold=0.5, name='iou'),
            keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy'),
            keras.metrics.Precision(thresholds=0.5, name='precision'),
            keras.metrics.Recall(thresholds=0.5, name='recall'),
            dice_coef,
            specificity
        ]

        # Compilar modelo com Focal Loss
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                    loss=combined_loss, 
                    metrics=metrics_to_track
                    )

        # Configurar callback para ajuste do learning rate
        lr_scheduler = ReduceLROnPlateau(
            monitor='val_iou',   # Monitorar o iou no conjunto de validação
            factor=0.5,          # Fator de redução da taxa de aprendizado
            patience=5,          # Número de épocas sem melhora antes de reduzir
            min_lr=1e-6,         # Limite inferior para o learning rate
            verbose=1
        )

        # Configurar callback para salvar modelo
        model_checkpoint = ModelCheckpoint(
            saving_dir+'/'+f'{model_name}.keras',
            monitor='val_iou',
            mode='max',
            save_best_only=True,
            verbose=1

        )

        # Configurar callback para early stopping
        early_stopping = EarlyStopping(
            monitor='val_iou',
            patience=10,
            restore_best_weights=True,
            verbose=1

        )

        # Treinar modelo
        print("Iniciando o treinamento...")
        print("Iniciando run do MLflow:", run.info.run_id)
        start_time = time.time()

        model.fit(
            train_dataset, 
            validation_data=val_dataset,  # Adiciona o conjunto de validação
            epochs=epochs, 
            verbose=1,
            callbacks=[lr_scheduler, model_checkpoint, early_stopping]
        )

    print(f"Treinamento concluído em {time.time() - start_time:.2f} segundos")
    print("Carregando o melhor modelo salvo do arquivo 'melhor_modelo.keras'...")

    # Não se esqueça de passar seus objetos customizados para que o Keras os reconheça!
    custom_objects = {
        'combined_loss': combined_loss, 
        'dice_loss': dice_loss, 
        'weighted_binary_crossentropy': weighted_binary_crossentropy,
        'iou': iou,
        'dice_coef': dice_coef,
        'specificity': specificity
    }

    # Carrega o melhor modelo que foi salvo durante o treinamento
    best_model = load_model(saving_dir+'/'+f'{model_name}.keras', custom_objects=custom_objects)

    print(f"Salvando o modelo no formato de pasta (TensorFlow SavedModel) em '{saving_dir}'...")

    # Salva o modelo novamente, mas desta vez sem extensão, para criar a pasta
    best_model.export(saving_dir+'/'+f'{model_name}')

    print("Processo concluído com sucesso!")

    return model

# Verificação dos dados de treinamento

In [33]:
# Definição completa da classe DataGenerator
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_dir, mask_dir, crop_size, batch_size, augment=False):
        self.image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir)])
        self.mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir)])
        self.crop_size = crop_size
        self.batch_size = batch_size
        self.augment = augment
        
        self.all_patches = self._create_patch_list()
        self.on_epoch_end()

    def _create_patch_list(self):
        patch_coords = []
        for i, img_path in enumerate(self.image_files):
            try:
                img_ds = gdal.Open(img_path)
                if img_ds is None:
                    print(f"Aviso: Não foi possível abrir a imagem {img_path}. Pulando.")
                    continue
                img_width, img_height = img_ds.RasterXSize, img_ds.RasterYSize
                
                for y in range(0, img_height - self.crop_size + 1, self.crop_size):
                    for x in range(0, img_width - self.crop_size + 1, self.crop_size):
                        patch_coords.append({'file_index': i, 'x': x, 'y': y})
            except Exception as e:
                print(f"Erro ao processar o arquivo {img_path}: {e}")
        return patch_coords

    def __len__(self):
        return int(np.floor(len(self.all_patches) / self.batch_size))

    def __getitem__(self, index):
        batch_patch_indices = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        patches_to_load = [self.all_patches[i] for i in batch_patch_indices]
        X, y = self._data_generation(patches_to_load)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.all_patches))
        np.random.shuffle(self.indexes)

    def _data_generation(self, patches_to_load):
        X = np.empty((self.batch_size, self.crop_size, self.crop_size, 3), dtype=np.float32)
        y = np.empty((self.batch_size, self.crop_size, self.crop_size, 1), dtype=np.float32)

        for i, patch_info in enumerate(patches_to_load):
            file_idx = patch_info['file_index']
            x, y_coord = patch_info['x'], patch_info['y']
            
            img_ds = gdal.Open(self.image_files[file_idx])
            mask_ds = gdal.Open(self.mask_files[file_idx])

            img_block = img_ds.ReadAsArray(x, y_coord, self.crop_size, self.crop_size)
            mask_block = mask_ds.ReadAsArray(x, y_coord, self.crop_size, self.crop_size)

            img_block = np.transpose(img_block, (1, 2, 0))
            if img_block.shape[2] == 4:
                img_block = img_block[:, :, :3]
            
            if mask_block.ndim == 2:
                mask_block = np.expand_dims(mask_block, axis=-1)

            img_block = (img_block / 255.0).astype(np.float32)
            mask_block = (mask_block).astype(np.float32)
            
            if self.augment:
                img_block, mask_block = augment_data(img_block, mask_block)

            X[i,] = img_block
            y[i,] = mask_block

        return X, y


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# --- Configure os seus parâmetros aqui ---
# (Use os mesmos valores que você usa para o treinamento)
crop_size = 128
batch_size = 8 # Pode ser qualquer valor, ex: 8, para visualização
augment = False # Mantenha como False para uma verificação limpa dos dados originais

# --- Fim da Configuração ---


# 1. Crie uma instância do seu DataGenerator
# (Assumindo que a classe DataGenerator que definimos anteriormente está disponível)
try:
    train_generator = DataGenerator(
        image_dir=train_image_dir,
        mask_dir=train_mask_dir,
        crop_size=crop_size,
        batch_size=batch_size,
        augment=augment
    )
except NameError:
    print("ERRO: A classe 'DataGenerator' não foi encontrada. Certifique-se de que ela está definida no seu script.")
    # Se der este erro, pare e cole a definição da classe DataGenerator aqui.
except FileNotFoundError:
    print(f"ERRO: Verifique se os caminhos '{train_image_dir}' e '{train_mask_dir}' estão corretos.")
    # Se der este erro, corrija os caminhos na seção de configuração acima.


# 2. Pega o primeiro lote de dados gerado
print("Gerando o primeiro lote de dados para visualização...")
images, masks = train_generator[0] 

# 3. Imprime informações de diagnóstico (Sanity Check)
print(f"Forma (shape) do lote de imagens: {images.shape}") # Deve ser (batch_size, 128, 128, 3)
print(f"Forma (shape) do lote de máscaras: {masks.shape}") # Deve ser (batch_size, 128, 128, 1)
print(f"Tipo de dados das imagens: {images.dtype}") # Deve ser float32
print(f"Tipo de dados das máscaras: {masks.dtype}") # Deve ser float32
print(f"Valores únicos na primeira máscara (deveria ser 0 e 1): {np.unique(masks[0])}")


# 4. Plota as imagens e máscaras
print("\nPlotando as imagens e máscaras...")
n_samples = min(5, batch_size)  # Mostra até 5 amostras do lote

plt.figure(figsize=(10, n_samples * 3))
for i in range(n_samples):
    # Plota a imagem original
    plt.subplot(n_samples, 2, 2 * i + 1)
    # A imagem foi normalizada (dividida por 255), então ela pode parecer escura ou estranha, isso é normal.
    plt.imshow(images[i])
    plt.title(f"Imagem de Treino {i}")
    plt.axis('off')

    # Plota a máscara correspondente
    plt.subplot(n_samples, 2, 2 * i + 2)
    # Usamos .squeeze() para remover a dimensão do canal (128, 128, 1) -> (128, 128)
    # Usamos cmap='gray' para garantir a visualização em preto e branco
    plt.imshow(masks[i].squeeze(), cmap='gray')
    plt.title(f"Máscara de Treino {i}")
    plt.axis('off')

plt.tight_layout()
plt.show()

# Criar experimento no MLFlow

In [ ]:
client = MlflowClient(tracking_uri='http://127.0.0.1:5000')

# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Projeto de detecção de crimes transfronteiriços na Amazônia. "
    "Este experimento contém os modelos de detecção de áreas com indícios de pistas de pouso ilegais"
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "nome_projeto":"Identificar Crimes Transfronteiriços na Amazônia",
    "tipo_crime": "Pistas de pouso",
    "mlflow.note.content": experiment_description

}

# Create the Experiment, providing a unique name
garimpos_experiment = client.create_experiment(
    name="Modelos_pistas", tags=experiment_tags
)

# Treinamento

In [ ]:
crop_size = 128
lr = 1e-3
epochs = 100
batch_size = 8
gamma = 2
augment=True

# Caminhos de dados
train_image_dir = "d:/SIMGEO 2025/DATASETS/GERAL/IMGS"
train_mask_dir = "d:/SIMGEO 2025/DATASETS/GERAL/MASKS"

model = train_model(train_image_dir=train_image_dir, 
                    train_mask_dir=train_mask_dir, 
                    epochs=epochs, 
                    crop_size=crop_size, 
                    batch_size=batch_size, 
                    lr=lr, 
                    gamma=gamma,
                    experiment_name='Modelos_gerais',
                    run_name='geral_teste1', 
                    model_name='modelo_melhor_treino_gerais', 
                    saving_dir='C:/Users/regin/Documents/ORF/ricardofranco/rede_neural_unet',
                    augment=augment)

# Predição

In [ ]:
def save_georeferenced_mask(predicted_mask, reference_image_path, output_path, threshold):

    # Abrir a imagem de referência para obter informações georreferenciadas
    ref_ds = gdal.Open(reference_image_path)
    if ref_ds is None:
        raise FileNotFoundError(f"Não foi possível abrir a imagem de referência: {reference_image_path}")
    
    # Obter informações georreferenciadas
    geo_transform = ref_ds.GetGeoTransform()
    projection = ref_ds.GetProjection()
    ref_ds = None  # Fechar o dataset da referência
    
    # Binarizar a máscara predita com um limite (threshold) de 0.5
    binary_mask = (predicted_mask >= threshold).astype(np.uint8)

    # Criar um arquivo TIFF georreferenciado
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(
        output_path, 
        binary_mask.shape[1], 
        binary_mask.shape[0], 
        1,  # Apenas um canal
        gdal.GDT_Byte  # Tipo de dado Byte (0 ou 1)
    )
    if out_ds is None:
        raise RuntimeError(f"Não foi possível criar o arquivo: {output_path}")
    
    # Aplicar informações georreferenciadas
    out_ds.SetGeoTransform(geo_transform)
    out_ds.SetProjection(projection)
    
    # Escrever a máscara binária no arquivo
    out_ds.GetRasterBand(1).WriteArray(binary_mask)
    
    # Fechar o dataset de saída
    out_ds.FlushCache()
    out_ds = None
    
    print(f"Máscara predita salva como TIFF georreferenciado em: {output_path}")

def crop_image(image_path, crop_size):
    # Abrir a imagem com GDAL
    img_ds = gdal.Open(image_path)
    if img_ds is None:
        raise FileNotFoundError(f"Não foi possível abrir a imagem: {image_path}")

    # Dimensões da imagem
    img_width = img_ds.RasterXSize
    img_height = img_ds.RasterYSize
    img_bands = img_ds.RasterCount

    # Inicializar listas para armazenar os blocos e coordenadas
    crops = []
    coords = []

    # Iterar sobre a imagem em blocos de crop_size x crop_size
    for y in range(0, img_height - crop_size + 1, crop_size):
        for x in range(0, img_width - crop_size + 1, crop_size):
            if img_bands > 3:  # Ajustar para imagens multibanda
                crop = np.stack(
                    [
                        img_ds.GetRasterBand(1).ReadAsArray(x, y, crop_size, crop_size),  # Banda 1 (R)
                        img_ds.GetRasterBand(2).ReadAsArray(x, y, crop_size, crop_size),  # Banda 2 (G)
                        img_ds.GetRasterBand(3).ReadAsArray(x, y, crop_size, crop_size),  # Banda 3 (B)
                    ],
                    axis=-1,  # Combinar no formato (H, W, C)
                )

            # Adicionar o bloco e suas coordenadas à lista
            crops.append(crop)
            coords.append((y, x))

    # Fechar o dataset GDAL
    img_ds = None

    return np.array(crops), coords

def reconstruct_image(predicted_crops, coords, original_shape, crop_size):
    # Criar matriz para reconstrução e mapa de contagem
    reconstructed = np.zeros(original_shape[:2], dtype=np.float32)
    count_map = np.zeros(original_shape[:2], dtype=np.float32)

    # Combinar blocos preditos na posição correta
    for (i, j), crop in zip(coords, predicted_crops):
        reconstructed[i:i+crop_size, j:j+crop_size] += crop.squeeze()
        count_map[i:i+crop_size, j:j+crop_size] += 1

    return reconstructed

def predict_and_save(test_imgs_dir, model, model_save_path, crop_size, threshold, save_model=False):

    imgs = os.listdir(test_imgs_dir)

    for img in imgs:
        if not img.endswith('.tif'):
            print(f'O arquivo {img} não é uma imagem TIF')
        else:
            test_img_path = os.path.join(test_imgs_dir, img)

            # Carregar e recortar a imagem de teste
            test_image_crops, test_coords = crop_image(test_img_path, crop_size=crop_size)
            test_image = cv2.imread(test_img_path) / 255.0
            
            # Prever cada bloco
            predicted_crops = model.predict(test_image_crops)

            # Reconstruir a máscara predita no tamanho original
            predicted_mask_full = reconstruct_image(predicted_crops, test_coords, test_image.shape, crop_size=crop_size)
            print(max(np.unique(predicted_mask_full)), min(np.unique(predicted_mask_full)))
            # Salvar a máscara predita
            predicted_mask_path = f"D:/ORF/dados-rf-cma/pista/resultados-testes/predicted_mask_{img}"
            save_georeferenced_mask(predicted_mask_full, test_img_path, predicted_mask_path, threshold=threshold)
    # if save_model == True:
    #     # Salvar o modelo treinado
    #     model.save(model_save_path+'.')
    #     print(f"Modelo treinado salvo em: {model_save_path}")

In [ ]:
predict_and_save(test_imgs_dir=test_image_path, model=model, model_save_path=model_save_path, crop_size=crop_size, threshold=0.75)